In [8]:
import numpy as np
import cPickle
import nltk
from nltk.corpus import stopwords
from collections import Counter
from sklearn.metrics.pairwise import pairwise_distances
import itertools
import string
from nltk.tokenize import sent_tokenize
from nltk.stem.porter import *
#import pyemd
#loading word embeddings
#wv = KeyedVectors.load_word2vec_format("../w2v/w2vemb.bin", binary=True)

# Visualize functions

In [9]:
def write_file(n_clusters_, labels, corpus, mapping_dict, test_corpus):
    for indice_cluster in range(n_clusters_):
        # print indice_cluster
        idx_list = np.where(labels == indice_cluster)[0]
        f1 = open('cluster_%s/%dth_cluster.txt' % (question_class, indice_cluster), 'w+')
        for idx in idx_list:
            f1.write('%s %s\n' % (corpus[mapping_dict[idx] - 1], test_corpus[idx]))
    f1.close()


def write_ne(nes, question_class, mapping_dict):
    f1 = open('ne_%s.txt' % question_class, 'w+')
    for idx, doc in enumerate(nes):
        f1.write('%s %s\n' % (mapping_dict[idx], ' '.join(doc)))
    f1.close()    

def wmd_compute(x1, x2):
    return wv.wmdistance(x1, x2)

def analyze_cluster(n_clusters_, labels, corpus, mapping_dict, test_corpus, filename):
    num = 0
    other_content = []
    for indice_cluster in range(n_clusters_):
        idx_list = np.where(labels == indice_cluster)[0]
        if len(idx_list) > 4:
            f1 = open('%s_%s/%dth_cluster.txt' % (filename, question_class, indice_cluster), 'w+')
            for idx in idx_list:
                f1.write('%s %s\n' % (corpus[mapping_dict[idx] - 1], test_corpus[idx]))
            f1.close()
        if len(idx_list) < 4:
            num = num + len(idx_list)
            for idx in idx_list:
                other_content.append(corpus[mapping_dict[idx] - 1])
    f1 = open('%s_%s/other_cluster.txt' % (filename, question_class), 'w+')
    for doc in other_content:
        f1.write('%s\n' % doc)
    f1.close()
    print num

# Reading Raw text

In [10]:
question_class = 'q3'
fname = '..//data//%s.txt' % question_class
with open(fname) as f:
    content = f.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line
content = [x.strip() for x in content]
print 'length of content', len(content)
f.close()
print content

length of content 309
['earlier , i had given my car to servicing they told me , \\? we will give complete details within 1 or 2 day or till monday \\? . after that they committed me that we will complete the servicing till friday handover car to you but it was not completed . they completed servicing on next friday . previously they checked problem and they were aware about my car door issue . after that they had not given me proper time frame . they should know how much time it will take to deliver the car back . they should say , \\? sir it will take 15 days , so please adjust to do your work . they finally gave our car on saturday .', 'they should explain everything', 'they should give proper explanation .', "they are not providing the correct information . they do n't have a capability to estimate what problem is there in the system .", 'after resolving the customer query customer should be satisfied after trial on car .', 'when customer come in showroom that time staff should qui

In [11]:
# split comment with multi-sentence into multi-comments
content_new = []
for comment in content:
    sents = sent_tokenize(comment)
    if len(sents) > 1:
        for i in range(len(sents)):
            content_new.append(sents[i])
    else:
        content_new.append(comment)
print 'length of new content', len(content_new)
# print content_new
    

length of new content 917


In [20]:
pos_new = nltk.pos_tag(nltk.word_tokenize("it is 10 km far away from city it should be nearby city within 2 3 km . "))
print pos_new

[('it', 'PRP'), ('is', 'VBZ'), ('10', 'CD'), ('km', 'NN'), ('far', 'RB'), ('away', 'RB'), ('from', 'IN'), ('city', 'NN'), ('it', 'PRP'), ('should', 'MD'), ('be', 'VB'), ('nearby', 'JJ'), ('city', 'NN'), ('within', 'IN'), ('2', 'CD'), ('3', 'CD'), ('km', 'NN'), ('.', '.')]


In [16]:
stemmer = PorterStemmer()
print stemmer.stem("vehicl")

vehicl


In [13]:

def rule_q1(sen, ne):
    clean_ne = list(set(ne))
#     remove_words = ["improv", "custom", "servic", "peopl","person","facil","avail","good",\
#                     "center","centr","car", "dealership", "vehicl", "toyota", "problem","work", "much",\
#                    "thing", "possibl","need"]   #stemmed
    remove_words = ['custom', 'car', 'vehicl','servic', '']
    clean_ne = [word for word in clean_ne if word not in remove_words]
    
    save_words = ["without", "call","wait", "pick", "pickup", "drop","remind", "inform", "respons", "book","fix",\
                 "receiv", "same", "sm", "immedi", "urgent","urgenc","deliv","deliveri","explain", "detail"]  #stemmed
    clean_ne = clean_ne + [stemmer.stem(word) for word in sen.split() if stemmer.stem(word) in save_words]
    
    # rules to merge keywords:
    
        
    clean_ne = list(set(clean_ne))
    return clean_ne

def clean_corpus(x1):
    x_new = []
    for x in x1:
        if 'no improvement' in x:
            continue
        else:
            x_new.append(x)
    return x_new

def process_corpus(x1, pos_tags, general_stop):
    # split three categories: 1 no improvemnt 2 with noun 3 others
    doc_noimprove = []
    doc_nn = []
    nn_extracted = []
    doc_other = []
    for x in x1:
        if 'no improvement' in x:
            doc_noimprove.append(x)
        else:
                nn_list = []
                sen = x
                pos_new = nltk.pos_tag(nltk.word_tokenize(sen))
                for token in pos_new:
                    if token[1] in pos_tags and not token[0] in general_stop:
                        nn_list.append(token[0])
                #stemming
                for counter,word in enumerate(nn_list):
                    nn_list[counter] = stemmer.stem(word)
                # apply rule
                #nn_list = rule_q1(sen, nn_list)
                    
                if nn_list != []:
                    nn_extracted.append(nn_list)
                    doc_nn.append(sen)
                else:
                    doc_other.append(sen)
    return doc_noimprove, [doc_nn, nn_extracted], doc_other
        
nn_corpus = []

# English stop words lists
stop_words = stopwords.words('english')
punctuation_list = [unicode(i) for i in string.punctuation]
for punctuation in punctuation_list:
    stop_words.append(punctuation)

pos_tags = ['NN', 'NNS','JJ','JJR','JJS', 'VB','VBD','VBG','VBN','VBP','VBZ']

doc1, doc2, doc3 = process_corpus(content_new, pos_tags, stop_words)

doc_nn, nn_extracted = doc2[0], doc2[1]
print 'signle sentence with nn', len(doc_nn)
print 'no comments', len(doc1)
print 'single sentence without nn', len(doc3)
print nn_extracted

signle sentence with nn 888
no comments 8
single sentence without nn 21
[['given', 'car', u'servic', 'told'], ['give', u'complet', u'detail', 'day', 'till', 'monday'], [u'commit', u'complet', u'servic', 'till', 'friday', u'handov', 'car', u'complet'], [u'complet', u'servic', 'next', 'friday'], [u'check', 'problem', u'awar', 'car', 'door', u'issu'], ['given', 'proper', 'time', 'frame'], ['know', 'much', 'time', 'take', u'deliv', 'car'], ['say'], ['sir', 'take', u'day', u'pleas', 'adjust', 'work'], ['gave', 'car', 'saturday'], ['explain', u'everyth'], ['give', 'proper', u'explan'], [u'provid', 'correct', u'inform'], [u'capabl', u'estim', 'problem', 'system'], [u'resolv', u'custom', u'queri', u'custom', u'satisfi', 'trial', 'car'], [u'custom', 'come', 'showroom', 'time', 'staff', 'respond', u'custom', 'staff', 'give', 'proper', u'time', u'inform', u'servic', 'car'], ['say', 'lie', u'custom', 'fact', 'say'], ['problem', u'solv', u'servic', 'center', 'need', 'visit', u'vehicl', u'manufactur

In [14]:
from collections import Counter
def df_count(x1):
    text = []
    for ab in x1:
        text = text + ab
    df = Counter(text)
    return df
df = df_count(nn_extracted)
print df

Counter({'car': 246, u'custom': 234, u'servic': 214, 'give': 115, 'staff': 115, 'problem': 109, 'work': 103, 'time': 99, u'inform': 89, 'need': 78, 'explain': 71, 'proper': 69, 'advisor': 68, u'part': 63, 'tell': 55, u'chang': 52, 'take': 51, 'told': 49, 'ask': 49, u'explan': 48, 'thing': 43, 'done': 43, 'know': 41, 'good': 40, u'check': 40, 'center': 38, u'vehicl': 37, u'detail': 36, 'dealership': 34, 'given': 33, u'improv': 32, u'issu': 30, u'provid': 29, 'come': 29, 'person': 26, u'complet': 25, u'want': 24, u'advis': 23, u'charg': 23, 'get': 23, 'said': 23, 'call': 22, 'talk': 22, 'go': 21, u'knowledg': 21, u'satisfi': 20, u'advic': 20, 'use': 18, 'understand': 18, 'answer': 17, 'toyota': 17, u'appoint': 17, 'visit': 16, u'anyth': 16, u'train': 15, 'say': 15, u'guid': 15, 'much': 15, 'small': 14, u'everyth': 14, 'drive': 14, u'avail': 14, 'new': 13, u'peopl': 13, 'wrong': 13, 'correct': 13, 'clear': 13, u'solv': 13, 'suggest': 13, 'taken': 12, 'first': 12, 'km': 12, u'requir': 12, 

In [15]:
import heapq
import os
def filter_rule_q1(doc, xth, test_corpus, original_corpus = doc_nn):
    apply_rule = False
    # print original_corpus[xth]
    # rules (priority from high to low)
    if 'app' in doc:
        test_corpus[xth] = ['app']
        print "rule_app applied"
        apply_rule = True
    elif 'without_appointment' in doc:
        test_corpus[xth] = ['without_appointment']
        print "rule_without_appointment applied"
        apply_rule = True
    elif 'sm' in doc:
        test_corpus[xth] = ['sm']
        print "rule_sm applied"
        apply_rule = True
    elif 'onlin' in doc:
        test_corpus[xth] = ['onlin']
        print "rule_onlin applied"
        apply_rule = True
    elif 'emerg' in doc:
        test_corpus[xth] = ['emerg']
        print "rule_emerg applied"
        apply_rule = True
    elif 'deliv' in doc:
        test_corpus[xth] = ['deliv']
        print "rule_deliv applied"
        apply_rule = True
    elif 'inform' in doc:
        test_corpus[xth] = ['inform']
        print "rule_inform applied"
        apply_rule = True
    elif 'fix' in doc:
        test_corpus[xth] = ['fix']
        print "rule_fix applied"
        apply_rule = True
    elif 'pick' in doc and 'drop' in doc:
        test_corpus[xth] = ['pick']
        print "rule_pick applied"
        apply_rule = True
    elif 'wait' in doc and 'month' not in original_corpus[xth]:
        test_corpus[xth] = ['wait']
        print "rule_wait applied"
        apply_rule = True
    elif 'appointment' in doc or 'day' in doc or 'week' in doc and 'call' in doc: # from call to appointment
        test_corpus[xth] = ['appointment']
        print "rule_appointment applied"
        apply_rule = True
        
    return apply_rule
    
def filter_ne(test_corpus, df):  # assuming each review contain one aspect
    for xth, doc in enumerate(test_corpus):
        if len(doc)>1:
            apply_rule = filter_rule_q1(doc, xth, test_corpus)
            if not apply_rule:
                df_words = [df[word] for word in doc]
                idx =  heapq.nlargest(1, xrange(len(df_words)), key=df_words.__getitem__)
                test_corpus[xth] = [doc[ith] for ith in idx]
    return test_corpus
nn_clean = filter_ne(nn_extracted, df)
df = df_count(nn_clean)
dict_map = dict(df.most_common())
print df

def write_file(corpus, idx_list, word):
    if not os.path.isdir("cluster/%s" %word):
            os.mkdir("cluster/%s" %word)
    f1 = open('cluster/%s/%s_comment.txt' % (word, word), 'w+')
    for idx in idx_list:
        f1.write('%s\n' %corpus[idx])
    f1.close()
    
    
def main_category(df_list, nn_clean, corpus):
    if (not os.path.isdir("cluster")):
        os.mkdir("cluster")
    name_list = {}
    clustered_index = []
    major_list = [word for word in df_list if df_list[word]>2]
    print "majot list:\n", major_list
    for word in major_list:
        idx_set = []
        for idx, doc in enumerate(nn_clean):
            if word in doc:
                idx_set.append(idx)
        write_file(doc_nn, idx_set, word)
        name_list[word] = idx_set
        clustered_index = clustered_index + idx_set
    clustered_index = set(clustered_index)
    print "clustered index:", list(clustered_index)
    unclustered_index = list(set(range(len(nn_clean))) - clustered_index)
    print "others index:", unclustered_index
    name_list['others'] = unclustered_index
    write_file(doc_nn, unclustered_index, 'others')
    return name_list

name_list = main_category(dict_map, nn_clean,doc_nn)
print name_list['call']

rule_appointment applied
rule_deliv applied
rule_appointment applied
rule_inform applied
rule_inform applied
rule_deliv applied
rule_inform applied
rule_fix applied
rule_inform applied
rule_inform applied
rule_inform applied
rule_inform applied
rule_inform applied
rule_inform applied
rule_inform applied
rule_inform applied
rule_inform applied
rule_inform applied
rule_inform applied
rule_inform applied
rule_inform applied
rule_inform applied
rule_inform applied
rule_inform applied
rule_inform applied
rule_inform applied
rule_inform applied
rule_wait applied
rule_appointment applied
rule_appointment applied
rule_inform applied
rule_appointment applied
rule_inform applied
rule_deliv applied
rule_inform applied
rule_deliv applied
rule_deliv applied
rule_wait applied
rule_inform applied
rule_inform applied
rule_deliv applied
rule_wait applied
rule_inform applied
rule_deliv applied
rule_inform applied
rule_inform applied
rule_inform applied
rule_inform applied
rule_wait applied
rule_inform a